In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy import stats as s
import statistics


In [2]:
from tqdm import tqdm

In [3]:
config = ConfigParser()
config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [4]:
connection_string

'DRIVER=FreeTDS;SERVER=172.16.34.15;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;'

In [6]:
# CODE FOR INSERTION IN AUDIT INTEGRATION 
cursor.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'REPORT_RISK',GETDATE(),'Error','RISK_REPORT','RISK_TRANSACTION_SUMMARY_STG',(SELECT MAX(T.MAX_DATE) FROM (select MAX(CREATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG UNION ALL select MAX(UPDATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG )T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock) where job_name = 'REPORT_RISK' ",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with(nolock)", conn)
conn.commit()


In [7]:
cursor.execute('''delete from  [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT] ''')
cursor.execute('''delete from  [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_rules] ''')
cursor.execute('''delete from  [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_summary_stg] ''')
cursor.execute('''delete from  [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] ''')
conn.commit()

In [8]:
# ALL THE DATA WILL BE CONSIDERED IN CASE OF THRESHOLD CALCULATION AND REPORT RISK TO BE RECALCULATED.
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] 
(
 REPORT_ID
 )
SELECT  DISTINCT REPORT_ID FROM [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_SUMMARY_STG with (nolock)''')
conn.commit()

In [9]:
# CALCULATING THE NUMBER OF RECOREDS TO BE CALCULATED.
no_of_records = pd.read_sql_query("SELECT COUNT(REPORT_ID) FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_INCREMENTAL WITH (NOLOCK)",conn)
no_of_records = no_of_records.iloc[0,0] 

In [10]:
# STORING VARIABLES USED TO UPDATE AUDIT INTGN.
%store job_id
%store no_of_records

Stored 'job_id' (int64)
Stored 'no_of_records' (int64)


In [11]:
no_of_records

182

In [12]:
cursor.execute('''
insert into FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG 
select DISTINCT BATCH_ID,REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE,GETDATE() AS CREATE_DATE,NULL AS UPDATE_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG
''')
conn.commit()

In [13]:
# # UPDATE DEBIT AMOUNT OF A REPORT
# tran_amnt_debit = '''UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG  SET TRAN_AMNT_DEBIT  = A.TRAN_AMNT_DEBIT
# FROM (
# SELECT  A.REPORT_ID,SUM(B.TRAN_AMNT) AS TRAN_AMNT_DEBIT
# FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG A
# INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN B
# ON A.REPORT_ID = B.REPORT_ID
# WHERE B.CR_DR_DEP_WITH IN ('DEBIT','Withdrawal','Buy','ATM Cash Withdrawal')
# GROUP BY A.REPORT_ID
# )a
# WHERE a.REPORT_ID = FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG.REPORT_ID'''

In [14]:
# # UPDATE CREDIT AMOUNT OF A REPORT
# tran_amnt_credit = '''
# UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG  SET TRAN_AMNT_CREDIT  = A.TRAN_AMNT_CREDIT
# FROM (
# SELECT  A.REPORT_ID,SUM(B.TRAN_AMNT) AS TRAN_AMNT_CREDIT
# FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG A
# INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN B
# ON A.REPORT_ID = B.REPORT_ID
# WHERE (B.CR_DR_DEP_WITH IN ('Credit','Deposit','Sell') or CR_DR_DEP_WITH  IS NULL)
# GROUP BY A.REPORT_ID
# )a
# WHERE a.REPORT_ID = FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG.REPORT_ID 
# '''

In [15]:
# cursor.execute(tran_amnt_credit)
# cursor.execute(tran_amnt_debit)
# conn.commit()

In [16]:
rules_df = pd.read_sql_query('''SELECT * FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK) WHERE RULE_NO LIKE 'R%' ''',conn)

In [17]:
rules_df = rules_df[rules_df['PERSON_TYPE'] == 'Report']

In [18]:
rules_df

RULE_ID     RULE_NO                                          RULE_DESC  \
5      5.25  R005025000  Donation of <amount> from a single entity is g...   
6      5.26  R005026000  High value transactions amounting to <amount> ...   
154    5.21  R005021000                  Cancellation of FCRA Registration   
155    5.22  R005022000                    Suspension of FCRA Registration   
156    5.23  R005023000                                   Black listed NGO   
157    5.24  R005024000                 Existence of FCRA account with NPO   
169    5.18  R005018000  High value funds  INR <> sent from the same pe...   
186    5.19  R005019000  High value funds INR <> received from the same...   
187     5.2  R005002000  Untraceable flags identified (E.g. true accoun...   
188    5.20  R005020000  High number <(count)> of transactions  to newl...   
197     5.1  R005001000  Any of reported parties already have trigerred...   
198    5.10  R005010000  High value cross border fund transfers amounti...   
199    5.11  R005011000  High value of transactions amounting to INR <a...   
200    5.12  R005012000  High value of transactions amounting to INR <a...   
201    5.14  R005014000                       Source of Funds is "Unknown"   
202    5.15  R005015000                  Destination of Funds is "Unknown"   
203    5.16  R005016000              Identity of the remitter is "Unknown"   
204    5.17  R005017000  <count of persons> entities/individuals as men...   
205    5.27  R005027000  Location of detection of counterfeit instrumen...   
206    5.28  R005028000  Notional/face value of the counterfeit currenc...   
207    5.29  R005029000  Deonomination Value <value> and Number of piec...   
208     5.3  R005003000  GoS tags reveals  high risk tags on any of ent...   
209    5.30  R005030000  <count of high risk report> of high risk trans...   
210     5.4  R005004000  High Risk Purpose code reported in CBWTR: <pur...   
211     5.5  R005005000  Value of transaction(s) INR <amount> reported ...   
212     5.6  R005006000  Number of transaction(s)  reported <COUNT>  is...   
213     5.7  R005007000  Number of involved entities/individuals <count...   
214     5.8  R005008000  Number of accounts <count> involved are greate...   
215     5.9  R005009000  Number of linked accounts <count> are greater ...   

    PERSON_TYPE         RISK_TYPE  IS_ACTIVE_FLAG CREATE_DATE UPDATE_DATE  \
5        Report       Report Risk               0  2022-12-07  1900-01-01   
6        Report       Report Risk               0  2022-12-07  1900-01-01   
154      Report       Report Risk               0  2022-12-07  1900-01-01   
155      Report       Report Risk               0  2022-12-07  1900-01-01   
156      Report       Report Risk               0  2022-12-07  1900-01-01   
157      Report       Report Risk               0  2022-12-07  1900-01-01   
169      Report       Report Risk               0  2022-12-07  1900-01-01   
186      Report       Report Risk               0  2022-12-07  1900-01-01   
187      Report  Auto High Rules                0  2022-12-07  1900-01-01   
188      Report       Report Risk               0  2022-12-07  1900-01-01   
197      Report  Auto High Rules                1  2022-12-07  1900-01-01   
198      Report       Report Risk               0  2022-12-07  1900-01-01   
199      Report       Report Risk               0  2022-12-07  1900-01-01   
200      Report       Report Risk               0  2022-12-07  1900-01-01   
201      Report       Report Risk               1  2022-12-07  1900-01-01   
202      Report       Report Risk               1  2022-12-07  1900-01-01   
203      Report       Report Risk               0  2022-12-07  1900-01-01   
204      Report       Report Risk               1  2022-12-07  1900-01-01   
205      Report       Report Risk               1  2022-12-07  1900-01-01   
206      Report       Report Risk               1  2022-12-07  1900-01-01   
207      Report       Report Risk             

In [19]:
# RUNNING REPORT RISK QUERIES TO POPULATE RISK_REPORT_RULES.

for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) ]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
#     print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    


('MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES WITH (HOLDLOCK) AS '
 'TARGET USING           (   SELECT    DISTINCT RTSSE.BATCH_ID,     '
 "RTSSE.REPORT_ID,       'R005001000' AS RULE_NO,   "
 'RPSS.ORGANIZATION_MASTER_ID AS ENTITY_ID ,     10 AS RISK_SCORE_STATIC,     '
 'GETDATE() AS CREATE_DATE , ENTITY_TYPE,PEER_GROUP_ID  FROM '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG RTSSE WITH '
 '(NOLOCK)   INNER JOIN '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES RPSS  WITH '
 '(NOLOCK)  ON RTSSE.ENTITY_ID=RPSS.ORGANIZATION_MASTER_ID   WHERE   '
 "RTSSE.ENTITY_TYPE = 'O'     AND RPSS.RULE_NO IN (  SELECT RULE_NO FROM "
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES  WITH (NOLOCK)  WHERE      '
 "RISK_TYPE='AUTO HIGH RULES ')       UNION     SELECT    DISTINCT            "
 "RTSSE.BATCH_ID,      RTSSE.REPORT_ID,        'R005001000' AS RULE_NO,        "
 'RPSS.PERSON_MASTER_ID AS ENTITY_ID,       10 AS RISK_SCORE_STATIC,       '
 'GETDATE() AS

Done
('MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES WITH '
 '(HOLDLOCK)     AS TARGET USING      (       SELECT  DISTINCT '
 "RTSST.BATCH_ID,    RTSST.REPORT_ID,    'R005028000' AS RULE_NO,      "
 'SUM(FC.TOTALAMOUNT) AS RULE_VALUE,    NULL AS RISK_SCORE,        GETDATE() '
 'AS CREATE_DATE ,ENTITY_ID,ENTITY_TYPE,PEER_GROUP_ID  FROM  '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG RTSST   WITH '
 '(NOLOCK)        INNER JOIN  FINCORE_DB.FINCORE.CPS_REPORT_DETAILS CRD WITH '
 '(NOLOCK)  ON CRD.FIU_BATCH_ID=RTSST.BATCH_ID AND '
 'CRD.REPORT_ID=RTSST.REPORT_ID         INNER JOIN '
 'FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_CCR FC WITH (NOLOCK) ON '
 'CRD.BATCH_ID=FC.BATCHID AND CRD.REPORT_ID=FC.REPORTID          INNER JOIN '
 'FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_COUNTERFEIT FCD WITH (NOLOCK) ON '
 'FC.ID_ = FCD.CCRID          INNER JOIN '
 'DATAHUB_DB.FIUMETAHUB.FINNET_CURRDENOMINATION DEN WITH (NOLOCK) ON '
 'FCD.CIDENOMINATION = DEN.DENOMINATIONVALUE     

Done


In [20]:

risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Report') ]['RULE_NO'].to_list()
auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ') & (risk_lkp['PERSON_TYPE'] == 'Report')]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Report')]['RULE_NO'].to_list()



In [21]:
auto_high_rules

['R005001000']

In [22]:
# Read RISK_REPORT_RULES for the static rules
if len(auto_high_rules) == 1:
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('{}')".format(auto_high_rules[0]),conn)
    
elif(len(auto_high_rules)== 0):
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('')",conn)
    
else:
    auto_high_rules = tuple(auto_high_rules)
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in" + str (auto_high_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [23]:
report_rules_auto_high = report_rules['REPORT_ID'].to_list()

In [24]:
report_rules_auto_high

[114614]

In [25]:
# for report in report_rules_auto_high:
#     print('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
#   select distinct  A.BATCH_ID,A.REPORT_ID,C.REPORT_TYPE,10 as RISK_SCORE,
#   GETDATE () AS CREATE_DATE, NULL AS RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
# on C.REPORT_ID = A.REPORT_ID where C.REPORT_ID = {}  '''.format(report_rules_auto_high) )
#     conn.commit()
# #     conn.close()

In [26]:
# INSERT DATA IN RISK_REPORT IF REPORT IS APPLICABLE IN A AUTO HIGH RULE

if len(report_rules_auto_high) == 1:
    
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,10 as RISK_SCORE,
GETDATE () AS CREATE_DATE,
NULL as RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID = {}
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high[0]))
    conn.commit()
    
    
    
elif len(report_rules_auto_high) > 1 :
    print("Populate risk_report for more than 1")
    report_rules_auto_high = tuple(report_rules_auto_high)
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,10 AS RISK_SCORE,
GETDATE() AS CREATE_DATE,
NULL as RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID  IN {} 
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE'''.format(report_rules_auto_high))
    conn.commit()
    

In [27]:
config = ConfigParser()
config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [28]:
# non_stat_rules = tuple(non_stat_rules)
stat_rules = tuple(static_rules)

In [29]:
# Read RISK_REPORT_RULES for the static rules
if len(non_stat_rules) == 1:
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    report_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [30]:
# REMOVING THE REPORTS THAT FALLS IN AUTO HIGH RULES
report_rules = report_rules[~report_rules['REPORT_ID'].isin(report_rules_auto_high)]

In [31]:
summary_stg = pd.read_sql_query('''select * from fincore_db.fincore_analytics.risk_report_summary_stg''',conn)

In [32]:
summary_stg = summary_stg[['REPORT_ID','REPORT_TYPE']]

In [33]:
peer_groups = pd.read_sql_query('''select distinct report_type from fincore_db.fincore_analytics.risk_report_summary_stg''',conn)

In [34]:
non_stat_rules

('R005001000',
 'R005028000',
 'R005029000',
 'R005005000',
 'R005006000',
 'R005007000',
 'R005008000')

In [35]:
report_rules = report_rules.merge(summary_stg,on = 'REPORT_ID',how = 'left')

In [36]:
# USED FOR THOSE REPORTS WHERE REPORT TYPE IS NULL.
report_rules['REPORT_TYPE'].fillna('UNKNOWN',inplace = True)

In [37]:
peer_groups = report_rules['PEER_GROUP_ID'].unique()
report_types = report_rules['REPORT_TYPE'].unique()
entity_types = ['I','O']

In [38]:
report_rules

BATCH_ID  REPORT_ID  ENTITY_ID ENTITY_TYPE        PEER_GROUP_ID  \
0      102901     100712          1           I  Government Sector_1   
1      102901     100712          1           I  Government Sector_1   
2      102901     100712          1           I  Government Sector_1   
3      102901     100712          1           I  Government Sector_1   
4      104102     102012          1           O       Not Declared_0   
..        ...        ...        ...         ...                  ...   
779    138203     135814        115           O     OTHERS_CHANGED_4   
780    141201     138712        116           O       Not Declared_0   
781    141201     138712        116           O       Not Declared_0   
782    141201     138712        116           O       Not Declared_0   
783    141201     138712        116           O       Not Declared_0   

        RULE_NO       RULE_VALUE  RISK_SCORE_STATIC  \
0    R005005000     565000000.00                NaN   
1    R005006000                1                NaN   
2    R005007000                1                NaN   
3    R005008000                1                NaN   
4    R005005000     152436322.00                NaN   
..          ...              ...                ...   
779  R005008000                1                NaN   
780  R005005000  238497289749.00                NaN   
781  R005006000                1                NaN   
782  R005007000                1                NaN   
783  R005008000                1                NaN   

                     CREATE_DATE REPORT_TYPE  
0    2023-05-22 15:49:18.1000000         STR  
1    2023-05-22 15:49:18.1300000         STR  
2    2023-05-22 15:49:18.1600000         STR  
3    2023-05-22 15:49:18.2700000         STR  
4    2023-05-22 15:49:18.1000000         STR  
..                           ...         ...  
779  2023-05-22 15:49:18.2700000         CTR  
780  2023-05-22 15:49:18.1000000         CTR  
781  2023-05-22 15:49:18.1300000         CTR  
782  2023-05-22 15:49:18.1600000         CTR  
783  2023-05-22 15:49:18.2700000         CTR  

[784 rows x 10 columns]

In [39]:
report_rules['RULE_VALUE'] = report_rules['RULE_VALUE'].astype('float64')

In [40]:
report_rules['RULE_VALUE'] = report_rules['RULE_VALUE'].astype('int64')

In [41]:
report_rules.drop_duplicates(inplace = True)

In [42]:
report_rules

BATCH_ID  REPORT_ID  ENTITY_ID ENTITY_TYPE        PEER_GROUP_ID  \
0      102901     100712          1           I  Government Sector_1   
1      102901     100712          1           I  Government Sector_1   
2      102901     100712          1           I  Government Sector_1   
3      102901     100712          1           I  Government Sector_1   
4      104102     102012          1           O       Not Declared_0   
..        ...        ...        ...         ...                  ...   
779    138203     135814        115           O     OTHERS_CHANGED_4   
780    141201     138712        116           O       Not Declared_0   
781    141201     138712        116           O       Not Declared_0   
782    141201     138712        116           O       Not Declared_0   
783    141201     138712        116           O       Not Declared_0   

        RULE_NO    RULE_VALUE  RISK_SCORE_STATIC                  CREATE_DATE  \
0    R005005000     565000000                NaN  2023-05-22 15:49:18.1000000   
1    R005006000             1                NaN  2023-05-22 15:49:18.1300000   
2    R005007000             1                NaN  2023-05-22 15:49:18.1600000   
3    R005008000             1                NaN  2023-05-22 15:49:18.2700000   
4    R005005000     152436322                NaN  2023-05-22 15:49:18.1000000   
..          ...           ...                ...                          ...   
779  R005008000             1                NaN  2023-05-22 15:49:18.2700000   
780  R005005000  238497289749                NaN  2023-05-22 15:49:18.1000000   
781  R005006000             1                NaN  2023-05-22 15:49:18.1300000   
782  R005007000             1                NaN  2023-05-22 15:49:18.1600000   
783  R005008000             1                NaN  2023-05-22 15:49:18.2700000   

    REPORT_TYPE  
0           STR  
1           STR  
2           STR  
3           STR  
4           STR  
..          ...  
779         CTR  
780         CTR  
781         CTR  
782         CTR  
783         CTR  

[784 rows x 10 columns]

# THRESHOLD CALCULATION

In [43]:
def not_normal_outliers_removal(d):
    print("not_normal_outliers_removal")
    data = [*set(d)]
    return compute_IQR(d,data)



In [44]:
def compute_IQR(d,data):
    print("compute_IQR")
    q3 = np.quantile(data, 0.75)
    q1 = np.quantile(data, 0.25)
    iqr = ((q3-q1)* 1.5)
    outliers_left = q1 - iqr
    outliers_right = q3 + iqr
    print("outliers_right",outliers_right)
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))
    data = list(filter(lambda x: x <= outliers_right,d))

    return data,outliers_right

In [45]:
def normal_outliers_removal(d):
    print("normal_outliers_removal")
    mean = np.mean(d)
    std = np.std(d)
    outliers_right = (mean + (3*std))
    data = list(filter(lambda x: x <= outliers_right, d))
    print(outliers_right)
    return data,outliers_right
    
    

In [46]:

def thresh_cal(d,outliers_right):
    print("thresh")
    median = np.median(d)
    mean = np.mean(d)
    std = np.std(d)
    mode = int(s.mode(d)[0])


    
    if std >= 1:
        one =   round(mean+std)
        two =   round(mean+2*std)
        three = round(mean+3*std)
        four =  round(mean+4*std)
        five =  round(mean+5*std)
        six =   round(mean+6*std)
        seven = round(mean+7*std)
        eight = round(mean+8*std)
        nine =  round(mean+9*std)
        ten =   round(mean+10*std)

    elif std > 0:
        one =   np.ceil(mean+std)
        two =   np.ceil(one+std)
        three = np.ceil(two+std)
        four =  np.ceil(three+std)
        five =  np.ceil(four+std)
        six =   np.ceil(five+std)
        seven = np.ceil(six+std)
        eight = np.ceil(seven+std)
        nine =  np.ceil(eight+std)
        ten =   np.ceil(nine+std)
        
        
    elif std == 0:
        one = np.ceil(mean+std+1)
        two = np.ceil(one+std+1)
        three = np.ceil(two+std+1)
        four = np.ceil(three+std+1)
        five = np.ceil(four+std+1)
        six = np.ceil(five+std+1)
        seven = np.ceil(six+std+1)
        eight = np.ceil(seven+std+1)
        nine =np.ceil(eight+std+1)
        ten = np.ceil(nine+std+1)

        
    print(one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right)

    return(one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right)




In [47]:

def compute_outlier(d):
    try:
    
        if (len(d) == 0):
            print("Not applicable")
            return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 

          

        elif ( len(d) < 5):
            print("Not applicable")
            return thresh_cal(d,"NA") 
            
        
        
        
        else:
            
            if ( len(d) <= 150):
                stats,p_val = shapiro(d)
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)   
                    
            else:
                stats,p_val  = kstest(d, 'norm')
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                    
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
    
            
    except KeyError:
        return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 
    
    return( None,None,None,None,None,None,None,None,None,None,None,None,None,None,None) 





In [48]:
peer_th = []
for peer_group in  peer_groups:
    temp = report_rules[report_rules['PEER_GROUP_ID'] == peer_group]
    for report_type in report_types:
        temp1 = temp[temp['REPORT_TYPE'] == report_type]
        for entity_type in entity_types:
            temp2 = temp1[temp1['ENTITY_TYPE'] == entity_type]
            for rule_no in non_stat_rules:
                data = temp2[temp2['RULE_NO'] == rule_no]
                data = data['RULE_VALUE'].to_list()
                data = list(map(int,data))

                one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right = compute_outlier(data)

                l =[peer_group,report_type,entity_type,rule_no,one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right]
                peer_th.append(l)

            

        

    




Not applicable
Not applicable
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 31614983.25
thresh
10356468 16947718 23538967 30130217 36721466 43312716 49903965 56495215 63086464 69677714 3765218.652173913 1050022.0 50000 6591249.535185175 31614983.25
not_normal_outliers_removal
compute_IQR
outliers_right 9.25
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.1886792452830188 1.0 1 0.9123182946216284 9.25
normal_outliers_removal
1.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 1.0
not_normal_outliers_removal
compute_IQR
outliers_right 2.5
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0754716981132075 1.0 1 0.2641509433962264 2.5
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 31614983.25
thresh
10356468 16947718 23538967 30130217 36721466 43312716 49903965 56495215 63086464 69677714 376

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


 10.0 11.0 1.0 1.0 1 0.0 1.0
not_normal_outliers_removal
compute_IQR
outliers_right 2.5
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0816326530612246 1.0 1 0.2738042421428314 2.5
Not applicable
Not applicable
Not applicable
Not applicable
thresh
3813789522 5644466118 7475142715 9305819312 11136495908 12967172504 14797849101 16628525698 18459202294 20289878890 1983112925.5 1983112925.5 152436329 1830676596.5 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.5 1.5 1 0.5 NA
Not applicable
Not applicable
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 1890041163968.75
thresh
704564479494 1063553220485 1422541961477 1781530702468 2140519443459 2499508184451 2858496925442 3217485666433 3576474407425 3935463148416 345575738502.7551 75400960002.0 758000000000 358988740991.3404 18900411639

1064886 1218409 1371931 1525454 1678976 1832499 1986021 2139544 2293066 2446589 911363.75 1000000.0 1000000 153522.4883923769 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.5 1.0 1 0.8660254037844386 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.5 1.0 1 0.8660254037844386 NA
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
1064886 1218409 1371931 1525454 1678976 1832499 1986021 2139544 2293066 2446589 911363.75 1000000.0 1000000 153522.4883923769 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.5 1.0 1 0.8660254037844386 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.5 1.0 1 0.8660254037844386 NA
Not applicable
Not a

In [49]:

# peer_th = []
# for peer_group in  peer_groups:
#     temp = report_rules[report_rules['REPORT_TYPE'] == peer_group]
#     for rule_no in non_stat_rules:
#         data = temp[temp['RULE_NO'] == rule_no]
#         data = data['RULE_VALUE'].to_list()
#         data = list(map(int,data))

#         one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right = compute_outlier(data)
        
#         l =[peer_group,rule_no,one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right]
#         peer_th.append(l)


In [50]:
# # CALCULATING THE THRESHOLD

# peer_th = []
# for peer_group in  peer_groups:
#     temp = report_rules[report_rules['REPORT_TYPE'] == peer_group]
#     for rule_no in non_stat_rules:
#         data = temp[temp['RULE_NO'] == rule_no]
#         data = data['RULE_VALUE'].to_list()
#         data = list(map(int,data))
#         median = np.median(data)
#         mean = np.mean(data)
#         std = np.std(data)
#         print(std)
#         if std >= 1:
#             print("std>1")
#             one = round(median+std)
#             two = round(median+2*std)
#             three = round(median+3*std)
#             four = round(median+4*std)
#             five = round(median+5*std)
#             six = round(median+6*std)
#             seven = round(median+7*std)
#             eight = round(median+8*std)
#             nine =round(median+9*std)
#             ten = round(median+10*std)

#         elif std > 0:
#             print("std>0")

#             one = np.ceil(median+std)
#             two = np.ceil(one+std)
#             three = np.ceil(two+std)
#             four = np.ceil(three+std)
#             five = np.ceil(four+std)
#             six = np.ceil(five+std)
#             seven = np.ceil(six+std)
#             eight = np.ceil(seven+std)
#             nine =np.ceil(eight+std)
#             ten = np.ceil(nine+std)


#         elif std == 0:
#             print("std=0")
#             one = np.ceil(median+std+1)
#             two = np.ceil(one+std+1)
#             three = np.ceil(two+std+1)
#             four = np.ceil(three+std+1)
#             five = np.ceil(four+std+1)
#             six = np.ceil(five+std+1)
#             seven = np.ceil(six+std+1)
#             eight = np.ceil(seven+std+1)
#             nine =np.ceil(eight+std+1)
#             ten = np.ceil(nine+std+1)
            
#         else:
#             one = 'NULL'
#             two = 'NULL'
#             three = 'NULL'
#             four = 'NULL'
#             five = 'NULL'
#             six = 'NULL'
#             seven = 'NULL'
#             eight = 'NULL'
#             nine ='NULL'
#             ten = 'NULL'
            


#         l =[peer_group,rule_no,one,two,three,four,five,six,seven,eight,nine,ten]
#         peer_th.append(l)





In [51]:
thresh = pd.DataFrame(data = peer_th,columns = ['PEER_GROUP_ID','REPORT_TYPE','ENTITY_TYPE','RULE_NO','ONE','TWO','THREE','FOUR','FIVE','SIX','SEVEN','EIGHT','NINE','TEN','MEAN','MEDIAN','MODE','STD','OUTLIER'])

In [52]:
thresh['PEER_GROUP_ID'].dropna(inplace = True)

In [53]:
thresh.fillna('NULL',inplace = True)

In [54]:
thresh.dropna(inplace = True)

In [55]:
config = ConfigParser()
config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [56]:
cursor.execute('''Delete from fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD  where rule_no like  'R%' ''')
conn.commit()

In [57]:
# INSERTING DATA IN THRESHOLD TABLE.

for ind,row in thresh.iterrows():

    cursor.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,MEAN,MEDIAN,MODE,STANDARD_DEVIATION,outlier,PEER_GROUP_ID,REPORT_TYPE,ENTITY_TYPE,RULE_NO)values({},{},{},{},{},{},{},{},{},{},{},{},{},{},'{}','{}','{}','{}','{}' )""".format(row['ONE'],row['TWO'],row['THREE'],row['FOUR'],row['FIVE'],row['SIX'],row['SEVEN'],row['EIGHT'],row['NINE'],row['TEN'],row['MEAN'],row['MEDIAN'],row['MODE'],row['STD'],row['OUTLIER'],row['PEER_GROUP_ID'],row['REPORT_TYPE'],row['ENTITY_TYPE'],row['RULE_NO']))
    conn.commit()


In [58]:
# # cALCULATING THE RISK SCORE OF REPORT FOR EVERY RULE.
# cursor.execute('''WITH CTE AS (
# select
# A.REPORT_ID,
# A.RULE_NO,
# case 
# 	when c.THRESHOLD1_1 IS null then 0
# 	when A.RULE_VALUE < c.threshold1_1 then 0 
# 	when A.RULE_VALUE = c.threshold1_1 then 1
# 	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
# 	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
# 	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
# 	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
# 	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
# 	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
# 	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
# 	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
# 	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
# 	else 10
# 	end RISK_SCORE,
# 	GETDATE() AS CREATE_DATE
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES a 
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG b 
# on a.REPORT_ID = b.REPORT_ID
# inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
# c.PEER_GROUP_ID = b.REPORT_TYPE and a.RULE_NO = c.RULE_NO
# where a.RISK_SCORE_STATIC is null
# )

# UPDATE  FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES SET RISK_SCORE_STATIC = CTE.RISK_SCORE
# FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# INNER JOIN CTE
# ON CTE.REPORT_ID = A.REPORT_ID AND CTE.RULE_NO = A.RULE_NO''')
# conn.commit()

In [59]:
cursor.execute('''WITH CTE AS (
select
A.REPORT_ID,
ENTITY_ID,
A.ENTITY_TYPE,
A.RULE_NO,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end RISK_SCORE,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES a 
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG b 
on a.REPORT_ID = b.REPORT_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = a.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO AND c.ENTITY_TYPE = a.ENTITY_TYPE AND b.REPORT_TYPE = c.REPORT_TYPE
inner join  FINCORE_DB.Fincore_Analytics.RISK_lkp_RULES d
on d.rule_no = a.rule_no
where risk_score_static_flag = 0
)

UPDATE  FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES SET RISK_SCORE_STATIC = CTE.RISK_SCORE
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
INNER JOIN CTE
ON CTE.REPORT_ID = A.REPORT_ID AND CTE.RULE_NO = A.RULE_NO AND A.ENTITY_ID = CTE.ENTITY_ID AND CTE.ENTITY_TYPE = A.ENTITY_TYPE ''' )
conn.commit()

In [60]:
weighted_avg = pd.read_sql_query('''SELECT BATCH_ID,REPORT_ID,
PRIORITY_NO,
CASE WHEN SUM(RULE_RISK)!= 0 THEN SUM(SQUARE(RULE_RISK))/SUM(RULE_RISK)
ELSE 0
END AS FINAL
FROM (
SELECT BATCH_ID,REPORT_ID,RULE_NO,AVG(RISK_SCORE_STATIC) AS RULE_RISK,PRIORITY_NO
FROM (
SELECT BATCH_ID,REPORT_ID,RRR.RULE_NO,ENTITY_ID,ENTITY_TYPE,RISK_SCORE_STATIC ,PRIORITY_NO 
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK) 
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES RRR WITH (NOLOCK) ON RLR.RULE_NO = RRR.RULE_NO
WHERE RLR.RISK_TYPE <> 'AUTO HIGH RULES ' 
)X
GROUP BY BATCH_ID,REPORT_ID,RULE_NO,PRIORITY_NO
)T
GROUP BY BATCH_ID,REPORT_ID,PRIORITY_NO ''',conn)


In [61]:
weights = pd.read_sql_query('''select distinct PRIORITY_NO,WEIGHTAGE from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES''',conn)

In [62]:
weights

PRIORITY_NO  WEIGHTAGE
0          NaN       0.05
1          NaN       0.50
2          1.0       0.50
3          2.0       0.30
4          3.0       0.15
5          4.0       0.05

In [63]:
weights.dropna(inplace = True)

In [64]:
weighted_avg = weighted_avg.merge(weights,on = 'PRIORITY_NO',how = 'inner')

In [65]:
weighted_avg = weighted_avg[~weighted_avg['REPORT_ID'].isin(report_rules_auto_high)]

In [66]:
weighted_avg

BATCH_ID  REPORT_ID  PRIORITY_NO      FINAL  WEIGHTAGE
0      102901     100712            1   9.111111        0.5
1      104102     102012            1   9.111111        0.5
2      104104     102013            1   9.111111        0.5
3      105201     103112            1   0.000000        0.5
4      105306     103216            1   0.000000        0.5
..        ...        ...          ...        ...        ...
177    137501     135112            1  10.000000        0.5
178    138101     135712            1   8.666667        0.5
179    138201     135812            1   0.000000        0.5
180    138203     135814            1  10.000000        0.5
181    141201     138712            1  10.000000        0.5

[181 rows x 5 columns]

In [67]:
weighted_avg['WEIGHTED_SCORE'] = weighted_avg['FINAL']*weighted_avg['WEIGHTAGE']

In [68]:
weighted_avg.drop(columns = ['PRIORITY_NO','FINAL'],inplace = True)

In [69]:
FINAL_DF = weighted_avg.groupby(['REPORT_ID']).sum()

In [70]:
FINAL_DF.reset_index(inplace = True)

In [71]:
risk_report = pd.read_sql_query('''Select REPORT_ID,REPORT_TYPE from fincore_db.fincore_analytics.RISK_REPORT_SUMMARY_STG''',conn)

In [72]:
FINAL_DF = FINAL_DF.merge(risk_report,how = 'inner', on = 'REPORT_ID')

In [73]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['WEIGHTED_SCORE']/FINAL_DF['WEIGHTAGE']

In [74]:
FINAL_DF

REPORT_ID  BATCH_ID  WEIGHTAGE  WEIGHTED_SCORE REPORT_TYPE  RISK_SCORE
0       100712    102901        0.5        4.555556         STR    9.111111
1       102012    104102        0.5        4.555556         STR    9.111111
2       102013    104104        0.5        4.555556         STR    9.111111
3       103112    105201        0.5        0.000000       CBWTR    0.000000
4       103216    105306        0.5        0.000000         NTR    0.000000
..         ...       ...        ...             ...         ...         ...
176     135112    137501        0.5        5.000000         CTR   10.000000
177     135712    138101        0.5        4.333333         CTR    8.666667
178     135812    138201        0.5        0.000000         STR    0.000000
179     135814    138203        0.5        5.000000         CTR   10.000000
180     138712    141201        0.5        5.000000         CTR   10.000000

[181 rows x 6 columns]

In [75]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['RISK_SCORE'].apply(lambda x: round(x))

In [76]:
count = 1
#UPDATING RISK_PERSON
for ind,row in tqdm(FINAL_DF.iterrows()):
    if count%10000 == 0:
        print(count,' HERE')
        conn.commit()
    count += 1

    conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_REPORT (BATCH_ID,REPORT_ID,REPORT_TYPE,RISK_SCORE_QUERY,CREATE_DATE)
    values ({},{},'{}',{},{})'''.format(row['BATCH_ID'],row['REPORT_ID'],row['REPORT_TYPE'],row['RISK_SCORE'],'Getdate()'))

conn.commit()

181it [00:00, 899.26it/s]


In [77]:
# # POPULATING THE RISK_REPORT 
# if len(report_rules_auto_high) == 0:
#     print("0")
#     cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
# select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
# GETDATE () AS CREATE_DATE,round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
# on C.REPORT_ID = A.REPORT_ID 
# inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
# on A.RULE_NO = B.RULE_NO
# GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE''')
#     conn.commit()
    
# elif len(report_rules_auto_high) == 1:
#     print(1)
    
#     cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
# select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
# GETDATE () AS CREATE_DATE,
# round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
# on C.REPORT_ID = A.REPORT_ID 
# inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
# on A.RULE_NO = B.RULE_NO
# where A.REPORT_ID <> {}
# GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high[0]))
#     conn.commit()
    
    
# else:
    
#     report_rules_auto_high = tuple(report_rules_auto_high)
#     print("more than 1")
#     cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
# select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
# GETDATE () AS CREATE_DATE,round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
# on C.REPORT_ID = A.REPORT_ID 
# inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
# on A.RULE_NO = B.RULE_NO
# where A.REPORT_ID NOT IN {} 
# GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high))
#     conn.commit()
    

In [78]:
conn.close()